In [1]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot

In [2]:
class FunctionHelper:
    def __init__(self, charMap, charCount, pairCount):
        self.charMap = charMap
        self.charCount = charCount
        self.pairCount = pairCount
        self.FINGERS_NAME = ['pinky', 'ring', 'middle', 'index', 'thumb', 'none', 'unknown']
        self.FINGERS = [0,1,2,3,4,5,6]
        self.FINGERS_WEIGHT = [3,2, 2, 1, 0, 0, 'Unknown']
        self.HAND = ['left', 'right', 'none', 'Unknown']
        self.COLUMNS = [0,1,2,3,'Unknown']
        self.COLUMNS_WEIGHT = [2,1,0,1.5,'Unknown']
        self.MOVEMENT = [0, 1, 2, 3, 'Unknown']

    def getCSVIndex(self,c):
        return self.charMap[c]
    
    def GetFinger(self,index):
        if(index >= 0 and index <= 9):
            return self.FINGERS[0]
        if(index >= 10 and index <= 17):
            return self.FINGERS[1]
        if(index >= 18 and index <= 25):
            return self.FINGERS[2]
        if(index >= 26 and index <= 41):
            return self.FINGERS[3]
        if(index >= 42 and index <= 55):
            return self.FINGERS[3]
        if(index >= 56 and index <= 63):
            return self.FINGERS[2]
        if(index >= 64 and index <= 71):
            return self.FINGERS[1]
        if(index >= 72 and index <= 93):
            return self.FINGERS[0]
        return self.FINGERS[-1]

    def GetHand(self, index):
        if(index >= 0 and index <= 41):
            return self.HAND[0]
        if(index >= 92 and index <= 93):
            return self.HAND[0]
        if(index >= 42 and index <= 91):
            return self.HAND[1]
        return self.HAND[-1]

    def ShiftHand(self, index):
        if(index%2 == 0):
            return self.HAND[2]
        if(self.GetHand(index) == self.HAND[0]):
            return self.HAND[1]
        if(self.GetHand(index) == self.HAND[1]):
            return self.HAND[0]
        return self.HAND[-1]

    def Column(self, index):
        if(index <= 85):
            ii = index%8
            if(ii == 0 or ii == 1):
                return self.COLUMNS[0]
            if(ii == 2 or ii == 3):
                return self.COLUMNS[1]
            if(ii == 4 or ii == 5):
                return self.COLUMNS[2]
            if(ii == 6 or ii == 7):
                return self.COLUMNS[3]
        
        if(index > 85):
            ii = index%8
            if(ii == 0 or ii == 1):
                return self.COLUMNS[1]
            if(ii == 2 or ii == 3):
                return self.COLUMNS[1]
            if(ii == 4 or ii == 5):
                return self.COLUMNS[0]
            if(ii == 6 or ii == 7):
                return self.COLUMNS[0]
        return self.COLUMNS[-1]

    def LateralMovement(self, index):
        index = index - (index%2)
        if(index == 0):
            return self.MOVEMENT[1]
        if(index == 92):
            return self.MOVEMENT[2]
        if(index == 90):
            return self.MOVEMENT[3]
        if(index >= 86 and index <= 88):
            return self.MOVEMENT[2]
        if(index >= 80 and index <= 84):
            return self.MOVEMENT[1]
        if(index >= 34 and index <= 46):
            return self.MOVEMENT[1]
        if(index >= 0 and index <= 93):
            return self.MOVEMENT[0]
        return self.MOVEMENT[-1]

    def WeightIndex(self, index):
        assert (index <= 93)
        finger = self.GetFinger(index)
        wFinger = self.FINGERS_WEIGHT[finger]
        lat = self.LateralMovement(index)
        colW = self.COLUMNS_WEIGHT[self.Column(index)]
        return ((lat+colW)**2)*wFinger 
    
    def LeftShift(self, indexA):
        ha = self.GetHand(indexA)
        hb = self.GetHand(6)
        if(ha != hb): return 0
        fa = self.GetFinger(indexA)
        cola = self.Column(indexA)
        lata = self.LateralMovement(indexA)
        fb = self.GetFinger(6)
        colb = self.Column(6)
        latb = self.LateralMovement(3)
        if(fa == fb):
            return ((cola-colb)**2+(lata-latb)**2) * 10 * self.FINGERS_WEIGHT[fa]
        return ((cola-colb)**2) * (self.FINGERS_WEIGHT[fa] + self.FINGERS_WEIGHT[fb])

    def RightShift(self, indexA):
        ha = self.GetHand(indexA)
        hb = self.GetHand(78)
        if(ha != hb): return 0
        fa = self.GetFinger(indexA)
        cola = self.Column(indexA)
        lata = self.LateralMovement(indexA)
        fb = self.GetFinger(78)
        colb = self.Column(78)
        latb = self.LateralMovement(78)
        if(fa == fb):
            return ((cola-colb)**2+(lata-latb)**2) * 10 * self.FINGERS_WEIGHT[fa]
        return ((cola-colb)**2) * (self.FINGERS_WEIGHT[fa] + self.FINGERS_WEIGHT[fb])

    def WeightPairWise(self, indexA, indexB):
        if(indexA%2 == 1 and indexB%2 == 1):
            iA = indexA-1
            iB = indexB-1
            ans = self.WeightPairWise(iA, iB)
            ha = self.GetHand(indexA)
            if(ha == self.HAND[1]):
                ans += self.LeftShift(iB)
            else:
                ans += self.RightShift(iB)
            
            hb = self.GetHand(indexB)
            if(hb == self.HAND[1]):
                ans += self.LeftShift(iA)
            else:
                ans += self.RightShift(iA)
            return ans
        if(indexA%2 == 1):
            iA = indexA-1
            iB = indexB
            ans = self.WeightPairWise(iA, iB)
            ha = self.GetHand(indexA)
            if(ha == self.HAND[1]):
                ans += self.LeftShift(iB)
            else:
                ans += self.RightShift(iB)
            return ans
        if(indexB%2 == 1):
            return self.WeightPairWise(indexB, indexA)


        ha = self.GetHand(indexA)
        hb = self.GetHand(indexB)
        if(ha != hb): return 0
        fa = self.GetFinger(indexA)
        cola = self.Column(indexA)
        lata = self.LateralMovement(indexA)
        fb = self.GetFinger(indexB)
        colb = self.Column(indexB)
        latb = self.LateralMovement(indexB)
        if(fa == fb):
            return ((cola-colb)**2+(lata-latb)**2) * 10 * self.FINGERS_WEIGHT[fa]
        return ((cola-colb)**2) * (self.FINGERS_WEIGHT[fa] + self.FINGERS_WEIGHT[fb])
    

In [3]:
def ObjectiveFunction(X, helper):
    ans = 0
    chars = 0
    XIndex = [helper.getCSVIndex(x) for x in X]
    for i, x in enumerate(XIndex):
        ans += helper.charCount[x]*helper.WeightIndex(i)
        chars += helper.charCount[x]
    for i,x in enumerate(XIndex):
        for j,y in enumerate(XIndex):
            ans += helper.pairCount[x][y]*helper.WeightPairWise(i,j)
            chars += helper.pairCount[x][y]
    return ans/chars
        

In [4]:
validChars = 'pPyYfFgGcCrRlLaAoOeEuUiIdDhHtTnNsSqQjJkKxXbBmMwWvVzZ$&[{}(=*)+]!#~%7531902468`;,./@\\:<>?^|-\'_"\t\n '
allChars = dict(zip(validChars,np.arange(len(validChars))))
charCountCSV = np.array(pd.read_csv('CharCount.csv').values)[:,2]
pairCountCSV = np.array(pd.read_csv('PairCount.csv').values)[:,1:]

In [5]:
helper = FunctionHelper(allChars, charCountCSV, pairCountCSV)

In [6]:
qwertyLayout = '1!qQaAzZ2@wWsSxX3#eEdDcC4$rRfFvV5%tTgGbB6^yYhHnN7&uUjJmM8*iIkK,<9(oOlL.>0)pP;:/?-_[{\'"=+]}\\|`~'
baseLayout = 'pPyYfFgGcCrRlLaAoOeEuUiIdDhHtTnNsSqQjJkKxXbBmMwWvVzZ$&[{}(=*)+]!#~%7531902468`;,./@\\:<>?^|-\'_"'
print(len(qwertyLayout))

94


In [7]:
funVal = ObjectiveFunction(qwertyLayout, helper)
print(funVal)

9.627145681198524


In [8]:
funVal = ObjectiveFunction(baseLayout, helper)
print(funVal)

15.212302700036597
